In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"trythse3k3r","key":"ceab88d17ac74170ecf8c829f0841a18"}'}

In [ ]:
!ls

kaggle.json  sample_data


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d robinreni/signature-verification-dataset


 97% 585M/601M [00:05<00:00, 92.5MB/s]
100% 601M/601M [00:05<00:00, 110MB/s] 


In [ ]:
from zipfile import ZipFile
file_name="signature-verification-dataset.zip"
with ZipFile(file_name,'r')as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import numpy as np 
import pandas as pd 
import os
%matplotlib inline
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import random
from PIL import Image
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from keras import backend as K
import cv2
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [ ]:
train_dir="/content/sign_data/train"
train_csv="/content/sign_data/train_data.csv"
test_csv="/content/sign_data/test_data.csv"
test_dir="/content/sign_data/test"

In [ ]:
def dataset_train(train_csvfile):
    x1=[]
    x2=[]
    y_train=[]
    for i in range(0,2000):
        image1_path=os.path.join(train_dir,train_csvfile.iat[i,0])
        image2_path=os.path.join(train_dir,train_csvfile.iat[i,1])
        img1=cv2.imread(image1_path)
        img1= cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        img1=cv2.resize(img1,(150,150))
        x1.append(img1)
        img2=cv2.imread(image2_path)
        img2= cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        img2=cv2.resize(img2,(150,150))
        x2.append(img2)
        y_train.append(train_csvfile.iat[i,2])
        print(i)
#         l.append([img1,img2,train_csvfile.iat[i,2]])
    x1=np.array(x1).astype(np.float32)/255.0
    x2=np.array(x2).astype(np.float32)/255.0
    y_train=np.array(y_train).astype(np.float32)
    
    return x1,x2,y_train

In [ ]:
def dataset_test(test_csvfile):
    x1=[]
    x2=[]
    y_train=[]
    for i in range(0,3000):
        image1_path=os.path.join(test_dir,test_csvfile.iat[i,0])
        image2_path=os.path.join(test_dir,test_csvfile.iat[i,1])
        img1=cv2.imread(image1_path)
        img1=cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        img1=cv2.resize(img1,(150,150))
        x1.append(img1)
        img2=cv2.imread(image2_path)
        img2=cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        img2=cv2.resize(img2,(150,150))
        x2.append(img2)
        y_train.append(test_csvfile.iat[i,2])
        print(i)
#         l.append([img1,img2,train_csvfile.iat[i,2]])
    x1=np.array(x1).astype(np.float32)/255.0
    x2=np.array(x2).astype(np.float32)/255.0
    y_train=np.array(y_train).astype(np.float32)
    
    return x1,x2,y_train

In [ ]:
df_test=pd.read_csv(test_csv)

In [ ]:
xt1,xt2,yt=dataset_test(df_test)

In [ ]:
df_train=pd.read_csv(train_csv)

In [ ]:
xs1,xs2,ys=dataset_train(df_train)

In [ ]:
def dist1(xy):
    x, y = xy
    sum_abs = K.sum(K.abs(x - y), axis=1, keepdims=True)
#     return sum_abs
    return K.maximum(sum_abs, K.epsilon())
def dist2(xy):
    x,y=xy
    sum_square=K.sum(K.square(x-y),axis=1,keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def dist3(xy):
    x,y=xy
    return K.sqrt(K.mean(K.square(x-y),axis=1,keepdims=True))
def dist4(xy):
    x,y=xy
    ss=K.sum(K.square(x-y),axis=1,keepdims=True)
    return K.sqrt(ss)/x.shape[1]


input1=keras.layers.Input(shape=(150,150,1))
# x=keras.layers.Flatten()(input1)
x=keras.layers.Conv2D(64,(10,10),activation='relu')(input1)
x=keras.layers.MaxPooling2D(2,2)(x)
x=keras.layers.Dropout(0.2)(x)
x=keras.layers.Conv2D(128,(7,7),activation='relu')(x)
x=keras.layers.MaxPooling2D(2,2)(x)
x=keras.layers.Conv2D(128,(4,4),activation='relu')(x)
x=keras.layers.MaxPooling2D(2,2)(x)
x=keras.layers.Dropout(0.5)(x)
 x=keras.layers.Conv2D(258,(4,4),activation='relu')(x)
x=keras.layers.MaxPooling2D(2,2)(x)
x=keras.layers.Flatten()(x)
x=keras.layers.Dense(4096,activation='relu')(x)
x=keras.layers.Dense(500,activation='relu')(x)
dense=keras.models.Model(inputs=input1,outputs=x)

img1=keras.layers.Input(shape=(150,150,1))
img2=keras.layers.Input(shape=(150,150,1))
dense1=dense(img1)
dense2=dense(img2)
fc=keras.layers.Lambda(dist3)([dense1,dense2])
fc=keras.layers.Dense(1,activation='sigmoid')(fc)

m=keras.models.Model(inputs=[img1,img2],outputs=fc)

m.compile(loss = "binary_crossentropy", optimizer="adam", metrics=["accuracy"])
m.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 1) 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 150, 150, 1) 0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 500)          69834164    input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1)            0           model[0][0]                

In [ ]:
m.fit([xs1,xs2],ys,epochs=16)

Epoch 1/16
63/63 [==============================] - 15s 108ms/step - loss: 0.6933 - accuracy: 0.5437
Epoch 2/16
63/63 [==============================] - 7s 105ms/step - loss: 0.6447 - accuracy: 0.5217
Epoch 3/16
63/63 [==============================] - 7s 105ms/step - loss: 0.5586 - accuracy: 0.5349
Epoch 4/16
63/63 [==============================] - 7s 105ms/step - loss: 0.4786 - accuracy: 0.5490
Epoch 5/16
63/63 [==============================] - 7s 106ms/step - loss: 0.4493 - accuracy: 0.5635
Epoch 6/16
63/63 [==============================] - 7s 106ms/step - loss: 0.4104 - accuracy: 0.6918
Epoch 7/16
63/63 [==============================] - 7s 106ms/step - loss: 0.3700 - accuracy: 0.8051
Epoch 8/16
63/63 [==============================] - 7s 107ms/step - loss: 0.3448 - accuracy: 0.9042
Epoch 9/16
63/63 [==============================] - 7s 107ms/step - loss: 0.3125 - accuracy: 0.9407
Epoch 10/16
63/63 [==============================] - 7s 107ms/step - loss: 0.3018 - accuracy: 0.962

In [ ]:
m.evaluate([xt1,xt2],yt)

94/94 [==============================] - 3s 33ms/step - loss: 0.4244 - accuracy: 0.8230


[0.4244031310081482, 0.8230000138282776]